# Tubes Mata Kuliah Kecerdasan Buatan

## kelompok 2 : "Prediksi Prestasi Akademik Siswa Menggunakan Machine Learning Berdasarkan Faktor Demografis dan Kehadiran"

Link Dataset: https://www.kaggle.com/datasets/waqi786/student-performance-dataset/data

Anggota :
1. Rakasya Yoga
2. R. Rafi Yudi Pramana
3. Datu reksa Hamza Putra
4. M. Ali Abdillah Khotami
5. Aditya

In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

Kode di atas digunakan untuk mengimport library-library yang dibutuhkan, seperti numpy untuk perhitungan numerik (array), pandas untuk dataframe, plt dan sns untuk visualisasi, dan sklearn untuk modelling

In [76]:
# df = pd.read_csv("../dataset/student_performance_data2.csv")
# df.head()

import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load Dataset
df = pd.read_csv("../dataset/xAPI-Edu-Data.csv")

# Kolom yang akan di-drop karena redundant & tidak memberi informasi baru
df = df.drop(columns=["PlaceofBirth"])

# Label Encoder untuk kolom biner & ordinal
label_cols = [
    "gender", "StageID", "GradeID", "Semester",
    "Relation", "ParentAnsweringSurvey",
    "ParentschoolSatisfaction", "StudentAbsenceDays"
]

le = LabelEncoder()
for col in label_cols:
    df[col] = le.fit_transform(df[col])

# One-Hot untuk Section ID (kecil & non-ordinal)
df = pd.get_dummies(df, columns=["SectionID"], drop_first=True)

# Frequency Encoding untuk kategori banyak
freq_cols = ["NationalITy", "Topic"]
for col in freq_cols:
    df[col] = df[col].map(df[col].value_counts())



In [75]:
print(df.head())

   gender  NationalITy  StageID  GradeID  Topic  Semester  Relation  \
0       1          179        2        1     95         0         0   
1       1          179        2        1     95         0         0   
2       1          179        2        1     95         0         0   
3       1          179        2        1     95         0         0   
4       1          179        2        1     95         0         0   

   raisedhands  VisITedResources  AnnouncementsView  Discussion  \
0           15                16                  2          20   
1           20                20                  3          25   
2           10                 7                  0          30   
3           30                25                  5          35   
4           40                50                 12          50   

   ParentAnsweringSurvey  ParentschoolSatisfaction  StudentAbsenceDays Class  \
0                      1                         1                   1     M   
1         

Kode di atas digunakan untuk membaca dataset yang akan digunakan, yaitu student_performance_data.csv. Kemudian 5 data teratas ditampilkan menggunakan head.

dalam data tersebut terdapat Kolom-kolom sebagai berikut:

1. sex: Jenis kelamin mahasiswa (0 = perempuan, 1 = laki-laki)  
2. Age: Usia mahasiswa  
3. StudyHoursPerWeek: Jumlah jam belajar per minggu  
4. AttendanceRate: Persentase kehadiran mahasiswa di kelas  
5. PartTimeJob: Status pekerjaan paruh waktu (0 = tidak bekerja, 1 = bekerja)  
6. ExtraCurricularActivities: Keterlibatan dalam kegiatan ekstrakurikuler (0 = tidak ikut, 1 = ikut)
7. GPA: Indeks prestasi kumulatif mahasiswa  



## Data Exploration

In [77]:
# FEATURE SELECTION
X = df.drop("Class", axis=1)  # hapus features
y = df["Class"]  # kelas label

# DATA SPLIT
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

# FEATURE SCALING
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [78]:
# MODEL TRAINING
# Naive Bayes
model_nb = GaussianNB()
model_nb.fit(X_train_scaled, y_train)
pred_nb = model_nb.predict(X_test_scaled)

# Decision Tree
model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(X_train, y_train)
pred_dt = model_dt.predict(X_test)

# SVM
model_svm = SVC(kernel='rbf', C=1, gamma='scale')
model_svm.fit(X_train_scaled, y_train)
pred_svm = model_svm.predict(X_test_scaled)

# Random Forest
model_rf = RandomForestClassifier(n_estimators=150, random_state=42)
model_rf.fit(X_train, y_train)
pred_rf = model_rf.predict(X_test)


## MODEL PERFORMANCE COMPARISON

In [79]:
# EVALUATION
models = {
    "Naive Bayes": pred_nb,
    "Decision Tree": pred_dt,
    "SVM": pred_svm,
    "Random Forest": pred_rf
}

print("\n====== MODEL PERFORMANCE COMPARISON ======\n")
for name, pred in models.items():
    print(f"📌 {name}")
    print("Accuracy:", round(accuracy_score(y_test, pred), 4))
    print(classification_report(y_test, pred))
    print("-" * 50)

# OPTIONAL: CONFUSION MATRIX
print("\nConfusion Matrices:")
for name, pred in models.items():
    cm = confusion_matrix(y_test, pred)
    print(f"\n{name}:\n{cm}")


====== MODEL PERFORMANCE COMPARISON ======

📌 Naive Bayes
Accuracy: 0.7292
              precision    recall  f1-score   support

           H       0.69      0.85      0.76        13
           L       0.69      0.92      0.79        12
           M       0.81      0.57      0.67        23

    accuracy                           0.73        48
   macro avg       0.73      0.78      0.74        48
weighted avg       0.75      0.73      0.72        48

--------------------------------------------------
📌 Decision Tree
Accuracy: 0.7917
              precision    recall  f1-score   support

           H       0.69      0.85      0.76        13
           L       0.80      1.00      0.89        12
           M       0.88      0.65      0.75        23

    accuracy                           0.79        48
   macro avg       0.79      0.83      0.80        48
weighted avg       0.81      0.79      0.79        48

--------------------------------------------------
📌 SVM
Accuracy: 0.6875
    

In [80]:
from sklearn.feature_selection import mutual_info_classif

info = mutual_info_classif(X, y, random_state=1)

info_fitur = pd.DataFrame({"Fitur": X.columns, "Informasi": info})
info_fitur = info_fitur.sort_values(by="Informasi")

print(info_fitur)

                       Fitur  Informasi
2                    StageID   0.000000
5                   Semester   0.000000
15               SectionID_C   0.000000
14               SectionID_B   0.000000
3                    GradeID   0.003331
4                      Topic   0.021040
6                   Relation   0.066099
0                     gender   0.071629
12  ParentschoolSatisfaction   0.077837
1                NationalITy   0.081584
10                Discussion   0.087161
11     ParentAnsweringSurvey   0.107493
9          AnnouncementsView   0.181143
8           VisITedResources   0.289487
13        StudentAbsenceDays   0.290832
7                raisedhands   0.304354


## Feature Selection

In [88]:
# FEATURE SELECTION
cols_to_drop = ["StageID", "Semester", "SectionID_C", "SectionID_B", "Class"]

X = df.drop(columns=cols_to_drop, axis=1)

# DATA SPLIT
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

# FEATURE SCALING
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [89]:
X.head()
from sklearn.feature_selection import mutual_info_classif

info = mutual_info_classif(X, y, random_state=1)

info_fitur = pd.DataFrame({"Fitur": X.columns, "Informasi": info})
info_fitur = info_fitur.sort_values(by="Informasi")

print(info_fitur)

                       Fitur  Informasi
3                      Topic   0.022220
2                    GradeID   0.034025
0                     gender   0.059025
1                NationalITy   0.065610
10  ParentschoolSatisfaction   0.075398
8                 Discussion   0.076117
9      ParentAnsweringSurvey   0.077814
4                   Relation   0.077984
7          AnnouncementsView   0.194704
5                raisedhands   0.275231
11        StudentAbsenceDays   0.287593
6           VisITedResources   0.287949


In [90]:
# MODEL TRAINING
# Naive Bayes
model_nb = GaussianNB()
model_nb.fit(X_train_scaled, y_train)
pred_nb = model_nb.predict(X_test_scaled)

# Decision Tree
model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(X_train, y_train)
pred_dt = model_dt.predict(X_test)

# SVM
model_svm = SVC(kernel='rbf', C=1, gamma='scale')
model_svm.fit(X_train_scaled, y_train)
pred_svm = model_svm.predict(X_test_scaled)

# Random Forest
model_rf = RandomForestClassifier(n_estimators=150, random_state=42)
model_rf.fit(X_train, y_train)
pred_rf = model_rf.predict(X_test)

# EVALUATION
models = {
    "Naive Bayes": pred_nb,
    "Decision Tree": pred_dt,
    "SVM": pred_svm,
    "Random Forest": pred_rf
}

print("\n====== MODEL PERFORMANCE COMPARISON ======\n")
for name, pred in models.items():
    print(f"📌 {name}")
    print("Accuracy:", round(accuracy_score(y_test, pred), 4))
    print(classification_report(y_test, pred))
    print("-" * 50)

# OPTIONAL: CONFUSION MATRIX
print("\nConfusion Matrices:")
for name, pred in models.items():
    cm = confusion_matrix(y_test, pred)
    print(f"\n{name}:\n{cm}")


====== MODEL PERFORMANCE COMPARISON ======

📌 Naive Bayes
Accuracy: 0.75
              precision    recall  f1-score   support

           H       0.73      0.85      0.79        13
           L       0.71      0.83      0.77        12
           M       0.79      0.65      0.71        23

    accuracy                           0.75        48
   macro avg       0.75      0.78      0.76        48
weighted avg       0.76      0.75      0.75        48

--------------------------------------------------
📌 Decision Tree
Accuracy: 0.7917
              precision    recall  f1-score   support

           H       0.73      0.85      0.79        13
           L       0.79      0.92      0.85        12
           M       0.84      0.70      0.76        23

    accuracy                           0.79        48
   macro avg       0.79      0.82      0.80        48
weighted avg       0.80      0.79      0.79        48

--------------------------------------------------
📌 SVM
Accuracy: 0.7292
      

In [58]:
# print("Naive Bayes")
# print("Accuracy:", round(accuracy_score(y_test, pred_nb), 4))
# print(classification_report(y_test, pred_nb))
# print("-" * 50)

# # CONFUSION MATRIX
# print("\nConfusion Matrices:")
# cm = confusion_matrix(y_test, pred_nb)
# print("Naive Bayes")
# print(cm)

# print("Random Forest")
# print("Accuracy:", round(accuracy_score(y_test, pred_rf), 4))
# print(classification_report(y_test, pred_rf))
# print("-" * 50)

# # CONFUSION MATRIX
# print("\nConfusion Matrices:")
# cm = confusion_matrix(y_test, pred_rf)
# print("Random Forest")
# print(cm)

# print("SVM")
# print("Accuracy:", round(accuracy_score(y_test, pred_svm), 4))
# print(classification_report(y_test, pred_svm))
# print("-" * 50)

# # CONFUSION MATRIX
# print("\nConfusion Matrices:")
# cm = confusion_matrix(y_test, pred_svm)
# print("SVM")
# print(cm)

# print("Decision Tree")
# print("Accuracy:", round(accuracy_score(y_test, pred_dt), 4))
# print(classification_report(y_test, pred_dt))
# print("-" * 50)

# # CONFUSION MATRIX
# print("\nConfusion Matrices:")
# cm = confusion_matrix(y_test, pred_dt)
# print("Decision Tree")
# print(cm)